# 0. 라이브러리 설치

In [ ]:
!pip install pycaret==2.3.10
!pip install jinja2==3.1.2
!pip install xgboost==1.6.0

     |████████████████████████████████| 320 kB 6.6 MB/s 
     |████████████████████████████████| 88 kB 8.9 MB/s 
     |████████████████████████████████| 1.3 MB 41.3 MB/s 
     |████████████████████████████████| 6.8 MB 31.6 MB/s 
     |████████████████████████████████| 56 kB 4.9 MB/s 
     |████████████████████████████████| 2.0 MB 44.8 MB/s 
     |████████████████████████████████| 167 kB 57.5 MB/s 
     |████████████████████████████████| 262 kB 55.4 MB/s 
     |████████████████████████████████| 118 kB 59.9 MB/s 
     |████████████████████████████████| 16.8 MB 18.0 MB/s 
     |████████████████████████████████| 1.7 MB 40.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 10.9 MB 42.9 MB/s 
     |████████████████████████████████| 690 kB 37.1 MB/s 
     |████████████████████████████████| 102 kB 12.3 MB/s 
     |████████████████

     |████████████████████████████████| 133 kB 7.2 MB/s 
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 193.7 MB 35 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


# 1. 데이터 불러오기

In [ ]:
import io
import pandas as pd

In [ ]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Likelion_Semi_project_03/data/train.csv')
df.drop(['index','occyp_type','FLAG_MOBIL'], axis = 1, inplace = True)
df.head()

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,0,1,0,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,0,0,0,2.0,-26.0,2.0


# 2. 데이터 전처리

In [ ]:
df.gender = df.gender.replace({'F' : 0, 'M' : 1})
df.car = df.car.replace({'N' : 0, 'Y' : 1})
df.reality = df.reality.replace({'N' : 0, 'Y' : 1})
df.head(3)

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,0,0,0,2.0,-6.0,1.0
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,0,0,1,3.0,-5.0,1.0
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,0,1,0,2.0,-22.0,2.0


In [ ]:
df['age'] = df.DAYS_BIRTH.apply(lambda x : -x // 365)
df.DAYS_EMPLOYED = (-1) * df.DAYS_EMPLOYED 
df.loc[(df.DAYS_EMPLOYED < 0), 'DAYS_EMPLOYED'] = 0
df.begin_month = (-1) * df.begin_month

In [ ]:
y = df.credit
X = df.drop(['DAYS_BIRTH'], axis = 1)

In [ ]:
X

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_EMPLOYED,work_phone,phone,email,family_size,begin_month,credit,age
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,4709,0,0,0,2.0,6.0,1.0,38
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,1540,0,0,1,3.0,5.0,1.0,31
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,4434,0,1,0,2.0,22.0,2.0,52
3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,2092,0,1,0,2.0,37.0,0.0,41
4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,2105,0,0,0,2.0,26.0,2.0,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,0,0,0,2,225000.0,State servant,Secondary / secondary special,Married,House / apartment,1984,0,0,0,4.0,2.0,1.0,33
26453,0,0,1,1,180000.0,Working,Higher education,Separated,House / apartment,2475,0,0,0,2.0,47.0,2.0,41
26454,0,1,0,0,292500.0,Working,Secondary / secondary special,Civil marriage,With parents,2015,0,0,0,2.0,25.0,2.0,27
26455,1,0,1,0,171000.0,Working,Incomplete higher,Single / not married,House / apartment,107,0,0,0,1.0,59.0,2.0,27


In [ ]:
# numeric_features = ['child_num', 'income_total', 'DAYS_EMPLOYED', 'family_size', 'begin_month', 'age']
# numeric_transformer = StandardScaler()

# categorical_features = ['income_type', 'edu_type', 'family_type', 'house_type']
# categorical_transformer = OneHotEncoder(categories='auto', handle_unknown = 'ignore')

# # label_features = ['credit']
# # label_transformer = LabelEncoder()

# preprocessor = ColumnTransformer(
#                 transformers=[
#                     ('num', numeric_transformer, numeric_features),
#                     ('cat', categorical_transformer, categorical_features)
#                 ], remainder='passthrough'
#                 )

In [ ]:
# preprocessor.fit(X)
# scaled_X = preprocessor.transform(X)

In [ ]:
# scaled_X

array([[-0.5735993 ,  0.1491363 ,  1.05922745, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.76452932,  0.59084783, -0.27784934, ...,  0.        ,
         0.        ,  1.        ],
       [-0.5735993 ,  2.57854975,  0.94319838, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.5735993 ,  1.03255937, -0.0774355 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.5735993 , -0.16006178, -0.88246627, ...,  0.        ,
         0.        ,  0.        ],
       [-0.5735993 , -1.04348485, -0.50020323, ...,  0.        ,
         0.        ,  0.        ]])

# 3. Pycaret 적용

In [ ]:
from pycaret.classification import *

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
model = setup(data= X,
              target='credit',
              numeric_features=['DAYS_EMPLOYED','child_num'], # categoric으로 분류돼서 직접 지정
              train_size=0.7,
              session_id=42)

,Description,Value
0,session_id,42
1,Target,credit
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(26457, 17)"
5,Missing Values,False
6,Numeric Features,6
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
# 평가 지표 추가
from sklearn.metrics import log_loss

add_metric('logloss','LogLoss',log_loss,greater_is_better=False,target='pred_proba')

Name                                                           LogLoss
Display Name                                                   LogLoss
Score Function                   <function log_loss at 0x7f9af0efdc20>
Scorer               make_scorer(log_loss, greater_is_better=False,...
Target                                                      pred_proba
Args                                                                {}
Greater is Better                                                False
Multiclass                                                        True
Custom                                                            True
Name: logloss, dtype: object

In [ ]:
# svm과 ridge 모델은 predict_proba를 지원하지 않아 제외
top_3_models = compare_models(sort='logloss', n_select=3, exclude=['svm','ridge'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss,TT (Sec)
xgboost,Extreme Gradient Boosting,0.7037,0.7089,0.4540,0.6851,0.6448,0.2770,0.3343,0.7650,6.957
lightgbm,Light Gradient Boosting Machine,0.7010,0.7017,0.4289,0.6994,0.6262,0.2428,0.3249,0.7658,0.968
gbc,Gradient Boosting Classifier,0.6924,0.6545,0.4087,0.6559,0.6075,0.2074,0.2966,0.7944,7.767
lda,Linear Discriminant Analysis,0.6450,0.6110,0.3375,0.5381,0.5119,0.0125,0.0510,0.8588,0.126
nb,Naive Bayes,0.6495,0.6074,0.3424,0.5787,0.5193,0.0279,0.0996,0.8643,0.042
lr,Logistic Regression,0.6435,0.5990,0.3333,0.4142,0.5040,-0.0001,-0.0014,0.8669,1.032
dummy,Dummy Classifier,0.6436,0.5000,0.3333,0.4142,0.5040,0.0000,0.0000,0.8792,0.034
rf,Random Forest Classifier,0.6985,0.7415,0.5315,0.6755,0.6802,0.3602,0.3678,0.9321,2.266
ada,Ada Boost Classifier,0.6922,0.6283,0.4039,0.6215,0.6028,0.1980,0.2987,1.0809,0.628
et,Extra Trees Classifier,0.6722,0.7048,0.5135,0.6497,0.6561,0.3124,0.3174,2.3828,2.423


In [ ]:
tuned_top3 = [tune_model(i, optimize='logloss') for i in top_3_models]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6960,0.6662,0.4134,0.6976,0.6127,0.2187,0.3088,0.7829
1,0.7003,0.6908,0.4242,0.7068,0.6228,0.2376,0.3223,0.7729
2,0.7046,0.6842,0.4288,0.7521,0.6277,0.2494,0.3379,0.7759
3,0.6884,0.6512,0.4033,0.6693,0.6008,0.1943,0.2843,0.7971
4,0.6922,0.6870,0.4117,0.6692,0.6097,0.2075,0.2960,0.7841
5,0.6928,0.6923,0.4136,0.6794,0.6117,0.2165,0.2963,0.7810
6,0.7014,0.6886,0.4210,0.6993,0.6207,0.2351,0.3278,0.7773
7,0.6879,0.6874,0.4039,0.6337,0.6021,0.1972,0.2798,0.7861
8,0.6949,0.6937,0.4133,0.6727,0.6124,0.2194,0.3039,0.7767


In [ ]:
tuned_top3

[XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
               colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
               importance_type=None, interaction_constraints='',
               learning_rate=0.3, max_bin=256, max_cat_to_onehot=4,
               max_delta_step=0, max_depth=8, max_leaves=0, min_child_weight=2,
               missing=nan, monotone_constraints='()', n_estimators=170,
               n_jobs=-1, num_parallel_tree=1, objective='multi:softprob',
               predictor='auto', random_state=42, reg_alpha=10, ...),
 LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, boosting_type='gbdt',
                class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
                importance_type='split', learning_rate=0.2, max_depth=-1,
                min_child_samples=6, min_child_wei

## 모델 블랜딩

In [ ]:
blend_top_3 = blend_models(estimator_list=tuned_top3, optimize='logloss')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.6976,0.6975,0.4174,0.6842,0.6166,0.2246,0.3146,0.7635
1,0.7014,0.7047,0.4242,0.6827,0.6238,0.2420,0.3254,0.7567
2,0.7025,0.7107,0.4291,0.7095,0.6271,0.2456,0.3306,0.7562
3,0.6906,0.6707,0.4124,0.7045,0.6090,0.2083,0.2907,0.7851
4,0.6976,0.7158,0.4181,0.7022,0.6167,0.2219,0.3156,0.7668
5,0.6976,0.7278,0.4225,0.6970,0.6204,0.2347,0.3124,0.7568
6,0.7030,0.7131,0.4267,0.6967,0.6258,0.2436,0.3325,0.7564
7,0.6938,0.6981,0.4159,0.6877,0.6138,0.2207,0.3000,0.7738
8,0.6965,0.7163,0.4170,0.7025,0.6158,0.2260,0.3094,0.7567


In [ ]:
# 블렌딩 모델로 예측
pred = predict_model(blend_top_3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.6958,0.7155,0.4265,0.6869,0.6174,0.2414,0.3264,0.7644


## 최종 모델 예측

In [ ]:
# finalize_model : 전체 데이터로 마지막 학습을 진행함
final_blended = finalize_model(blend_top_3)

In [ ]:
# 최종 모델로 예측
predict_model(final_blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
0,Voting Classifier,0.7106,0.8716,0.4477,0.7403,0.6387,0.2828,0.3745,0.6602


,child_num,income_total,DAYS_EMPLOYED,family_size,begin_month,age,gender_0,car_0,reality_0,income_type_Commercial associate,...,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents,work_phone_1,phone_0,email_0,credit,Label,Score
0,0.0,315000.0,545.0,1.0,11.0,28.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2,0.6656
1,1.0,216000.0,1782.0,2.0,7.0,28.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2,0.6142
2,0.0,238500.0,7095.0,2.0,53.0,41.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0,2,0.6968
3,0.0,112500.0,0.0,2.0,14.0,66.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2,0.6613
4,1.0,247500.0,1931.0,3.0,16.0,35.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.6631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7933,0.0,135000.0,0.0,1.0,-0.0,63.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1,0.5305
7934,0.0,270000.0,1674.0,2.0,29.0,26.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1,0.5535
7935,0.0,135000.0,1259.0,2.0,24.0,28.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2,0.5684
7936,0.0,117000.0,0.0,2.0,39.0,63.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,2,0.7374
